In [1]:
#import requests
import time
import json
import pandas as pd
import csv
from collections import Counter
from datetime import datetime
import requests
import unicodedata
import os
from datetime import datetime, timedelta


In [2]:
# Save files in separate directory
SAVE_DIR = "Saved_files"
os.makedirs(SAVE_DIR, exist_ok=True)

## Functions to define the search and respect rate limits (omitting by splitting search into FoS and year-by-year, month-by-mont)

In [8]:
from datetime import datetime, timedelta
import time
import calendar
import requests

BASE_URL = "https://api.semanticscholar.org/graph/v1"

def format_semantic_scholar_query(mandatory_keywords, optional_keywords):
    mandatory_query = ' '.join(f'"{keyword}"' for keyword in mandatory_keywords)
    optional_query = ' '.join(f'|"{keyword}"' for keyword in optional_keywords)
    return f"{mandatory_query} {optional_query}".strip()

"""
def search_papers(query, year, field_of_study=None, limit=100, offset=0, month_start=None, month_end=None):
    print(f"Searching papers for year {year}, field of study {field_of_study}, offset: {offset}")
    url = f"{BASE_URL}/paper/search"
    
    # Format dates according to API syntax
    if month_start is not None:
        # Calculate last day of month
        if month_start in [4, 6, 9, 11]:
            last_day = 30
        elif month_start == 2:
            # Simplified leap year check
            last_day = 29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28
        else:
            last_day = 31
            
        date_range = f"{year}-{month_start:02d}-01:{year}-{month_start:02d}-{last_day}"
        print(f"Using date range: {date_range}")
    
    params = {
        "query": query,
        "fields": "paperId,title,abstract,authors,year,citationCount,fieldsOfStudy,venue",
        "limit": limit,
        "offset": offset
    }
    
    if month_start is not None:
        params["publicationDateOrYear"]=date_range
    else:
        params["year"]=year
        
    if field_of_study:
        params["fieldsOfStudy"] = field_of_study
    #print (params) #only for debugging
    try:
        response = requests.get(url, params=params, timeout=30)
        if response.status_code in [429,504]:  # Rate limit hit
            retry_after = int(response.headers.get('Retry-After', 10))
            print(f"Rate limited. Waiting {retry_after} seconds...")
            time.sleep(retry_after)
            return search_papers(query, year, field_of_study, limit, offset, month_start,month_end) #calling the function to start again
            
        response.raise_for_status()
        data = response.json()
        papers = data.get("data", [])
        total = data.get("total", 0)
        print(f"Number of papers retrieved: {len(papers)}")
        return papers, total
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while searching papers: {e}")
        return [], 0
"""    

def search_papers(query, year, field_of_study=None, limit=100, offset=0, 
                  search_month_separately=False, month_start=None, month_end=None):
    """
    Search papers for a given year (optionally split by month).
    If search_month_separately is True, retrieves papers for each month in the year.
    Otherwise, retrieves all papers for the year or for a specific month range.
    """
    import calendar
    all_papers = []
    total_results = 0

    def get_last_day(year, month):
        return calendar.monthrange(year, month)[1]

    if search_month_separately:
        months = range(1, 13) if month_start is None or month_end is None else range(month_start, month_end+1)
        for month in months:
            last_day = get_last_day(year, month)
            date_range = f"{year}-{month:02d}-01:{year}-{month:02d}-{last_day}"
            print(f"Searching for {year}-{month:02d}, date range: {date_range}")
            params = {
                "query": query,
                "fields": "paperId,title,abstract,authors,year,citationCount,fieldsOfStudy,venue",
                "limit": limit,
                "offset": offset,
                "publicationDateOrYear": date_range
            }
            if field_of_study:
                params["fieldsOfStudy"] = field_of_study
            try:
                response = requests.get(f"{BASE_URL}/paper/search", params=params, timeout=30)
                if response.status_code in [429, 504]:
                    retry_after = int(response.headers.get('Retry-After', 10))
                    print(f"Rate limited. Waiting {retry_after} seconds...")
                    time.sleep(retry_after)
                    # Retry the same month
                    response = requests.get(f"{BASE_URL}/paper/search", params=params, timeout=30)
                response.raise_for_status()
                data = response.json()
                papers = data.get("data", [])
                total = data.get("total", 0)
                print(f"Month {month}: {len(papers)} papers retrieved.")
                all_papers.extend(papers)
                total_results += total
            except requests.exceptions.RequestException as e:
                print(f"An error occurred for month {month}: {e}")
                continue
        return all_papers, total_results
    else:
        # Whole year or specified range
        if month_start is not None and month_end is not None:
            first_day = 1
            last_day = get_last_day(year, month_end)
            date_range = f"{year}-{month_start:02d}-01:{year}-{month_end:02d}-{last_day}"
            print(f"Using date range: {date_range}")
            params = {
                "query": query,
                "fields": "paperId,title,abstract,authors,year,citationCount,fieldsOfStudy,venue",
                "limit": limit,
                "offset": offset,
                "publicationDateOrYear": date_range
            }
        else:
            params = {
                "query": query,
                "fields": "paperId,title,abstract,authors,year,citationCount,fieldsOfStudy,venue",
                "limit": limit,
                "offset": offset,
                "year": year
            }
        if field_of_study:
            params["fieldsOfStudy"] = field_of_study
        try:
            response = requests.get(f"{BASE_URL}/paper/search", params=params, timeout=30)
            if response.status_code in [429, 504]:
                retry_after = int(response.headers.get('Retry-After', 10))
                print(f"Rate limited. Waiting {retry_after} seconds...")
                time.sleep(retry_after)
                response = requests.get(f"{BASE_URL}/paper/search", params=params, timeout=30)
            response.raise_for_status()
            data = response.json()
            papers = data.get("data", [])
            total = data.get("total", 0)
            print(f"Number of papers retrieved: {len(papers)}")
            return papers, total
        except requests.exceptions.RequestException as e:
            print(f"An error occurred while searching papers: {e}")
            return [], 0
        


def get_last_day(year, month):
    return calendar.monthrange(year, month)[1]

def is_min_interval(date_start, date_end, min_interval='month'):
    start = datetime.strptime(date_start, "%Y-%m-%d")
    end = datetime.strptime(date_end, "%Y-%m-%d")
    if min_interval == 'month':
        return (start.year == end.year and start.month == end.month)
    elif min_interval == 'day':
        return (start == end)
    return False

def search_papers_recursive(query, year, field_of_study=None, limit=100, offset=0,
                            date_start=None, date_end=None, min_interval='month', max_retries=3,
                            visited=None, avg_query_time=10, queries_left_hint=None):
    if visited is None:
        visited = set()
    all_papers = []
    total_results = 0

    # Prevent infinite recursion by tracking visited ranges
    key = (date_start, date_end, offset)
    if key in visited:
        print(f"Already visited {date_start} to {date_end} offset {offset}, skipping to avoid infinite loop.")
        return all_papers, total_results
    visited.add(key)

    # Determine interval type
    if date_start and date_end:
        start_dt = datetime.strptime(date_start, "%Y-%m-%d")
        end_dt = datetime.strptime(date_end, "%Y-%m-%d")
        delta_days = (end_dt - start_dt).days + 1
        if delta_days > 183:
            interval = 'year'
        elif delta_days > 92:
            interval = '6month'
        elif delta_days > 31:
            interval = '3month'
        else:
            interval = 'month'
    else:
        interval = 'year'
        date_start = f"{year}-01-01"
        date_end = f"{year}-12-31"

    params = {
        "query": query,
        "fields": "paperId,title,abstract,authors,year,citationCount,fieldsOfStudy,venue",
        "limit": limit,
        "offset": offset,
        "publicationDateOrYear": f"{date_start}:{date_end}"
    }
    if field_of_study:
        params["fieldsOfStudy"] = field_of_study

    # Print expected remaining time estimate
    if queries_left_hint is not None:
        est_time = queries_left_hint * avg_query_time
        print(f"Estimated remaining time: {est_time/60:.1f} minutes for {queries_left_hint} queries...")

    for attempt in range(max_retries):
        try:
            t0 = time.time()
            response = requests.get(f"{BASE_URL}/paper/search", params=params, timeout=30)
            if response.status_code == 429:
                retry_after = int(response.headers.get('Retry-After', 10))
                print(f"Rate limited. Waiting {retry_after} seconds...")
                time.sleep(retry_after)
                continue
            response.raise_for_status()
            data = response.json()
            papers = data.get("data", [])
            total = data.get("total", 0)
            print(f"Date range {date_start} to {date_end} (offset={offset}): {len(papers)} papers retrieved.")
            all_papers.extend(papers)
            total_results += total
            t1 = time.time()
            avg_query_time = (avg_query_time + (t1-t0))/2  # update average

            # If we hit the limit, split further if possible
            if len(papers) == limit:
                if not is_min_interval(date_start, date_end, min_interval):
                    print(f"Splitting date range {date_start} to {date_end} into smaller intervals...")
                    splits = []
                    if interval == 'year':
                        splits = [(f"{year}-01-01", f"{year}-06-30"), (f"{year}-07-01", f"{year}-12-31")]
                    elif interval == '6month':
                        splits = [
                            (f"{year}-01-01", f"{year}-03-31"),
                            (f"{year}-04-01", f"{year}-06-30"),
                            (f"{year}-07-01", f"{year}-09-30"),
                            (f"{year}-10-01", f"{year}-12-31"),
                        ]
                    elif interval == '3month':
                        for m in range(1, 13):
                            m_start = f"{year}-{m:02d}-01"
                            m_end = f"{year}-{m:02d}-{get_last_day(year, m)}"
                            if m_start >= date_start and m_end <= date_end:
                                splits.append((m_start, m_end))
                    for s_start, s_end in splits:
                        sub_papers, _ = search_papers_recursive(
                            query, year, field_of_study, limit, 0,
                            date_start=s_start, date_end=s_end, min_interval=min_interval,
                            visited=visited, avg_query_time=avg_query_time
                        )
                        all_papers.extend(sub_papers)
                    return all_papers, total_results
                else:
                    # At minimum interval, use offset pagination
                    print(f"At minimum interval ({date_start} to {date_end}), paginating with offsets...")
                    page = 1
                    while True:
                        next_offset = page * limit
                        params["offset"] = next_offset
                        t0 = time.time()
                        response = requests.get(f"{BASE_URL}/paper/search", params=params, timeout=30)
                        if response.status_code == 429:
                            retry_after = int(response.headers.get('Retry-After', 10))
                            print(f"Rate limited. Waiting {retry_after} seconds...")
                            time.sleep(retry_after)
                            continue
                        response.raise_for_status()
                        data = response.json()
                        next_papers = data.get("data", [])
                        print(f"Paginated offset {next_offset}: {len(next_papers)} papers retrieved.")
                        all_papers.extend(next_papers)
                        t1 = time.time()
                        avg_query_time = (avg_query_time + (t1-t0))/2
                        if len(next_papers) < limit:
                            break
                        page += 1
            return all_papers, total_results
        except requests.exceptions.RequestException as e:
            print(f"An error occurred while searching papers: {e}")
            time.sleep(5)
    print(f"Failed to retrieve papers for {date_start} to {date_end} after {max_retries} attempts.")
    return all_papers, total_results


def iterate_fields_of_study(mandatory_keywords, optional_keywords, fields_of_study, start_year=1980, end_year=None):
    semantic_search_query_string = format_semantic_scholar_query(mandatory_keywords, optional_keywords)
    all_papers = []
    if end_year is None: 
        end_year = datetime.now().year
    papers_since_last_save = 0
    save_interval = 10

    # Try to load existing results
    try:
        filename = os.path.join(SAVE_DIR, "partial_results.json")
        with open(filename, 'r', encoding='utf-8') as f:
            all_papers = json.load(f)
        print(f"Loaded {len(all_papers)} existing results")
    except FileNotFoundError:
        print("Starting fresh search")
    except UnicodeDecodeError:
        print("Error reading existing file, starting fresh search")
        all_papers = []

    for year in range(start_year, end_year + 1):
        print(f"\nSearching for year {year}")
        for field in fields_of_study:
            print(f"Searching for field of study: {field}")
            try:
                # Use recursive search with date splitting and pagination
                papers, total = search_papers(
                    semantic_search_query_string,
                    year,
                    field,
                    limit=100,
                    search_month_separately=True  # Split by month for more granular results
                )
                if not papers:
                    continue
                for paper in papers:
                    if not is_duplicate(paper, all_papers):
                        all_papers.append(paper)
                        papers_since_last_save += 1
                # Save periodically
                if papers_since_last_save >= save_interval:
                    save_results_to_file(all_papers)
                    papers_since_last_save = 0
                    print(f"Saved {len(all_papers)} total papers")
            except Exception as e:
                print(f"Error during search: {e}")
                save_results_to_file(all_papers)
                time.sleep(5)
                continue
    save_results_to_file(all_papers)
    return all_papers




def save_results_to_file(results, filename="partial_results.json"):
#Save current results to a JSON file with cleaning
    cleaned_results = [clean_paper_data(paper) for paper in results]
    filepath= os.path.join(SAVE_DIR,filename) # setting the filepath to the assigned save directory, using the input filename.
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(cleaned_results, f, ensure_ascii=False, indent=2)
    print(f"Saved {len(results)} results to {filepath}")

In [4]:
""" no longer used"""

def search_papers_with_pagination(query, year, field_of_study=None, offset=0, max_papers=10000, month_start=None, month_end=None):
    """Search papers with pagination support"""
    all_papers = []
    total_retrieved = 0
    
    while total_retrieved < max_papers:
        try:
            papers, total = search_papers(
                query, 
                year, 
                field_of_study, 
                month_start=month_start,
                month_end=month_end,
                limit=100, 
                offset=offset)
            if not papers:
                break
                
            all_papers.extend(papers)
            total_retrieved += len(papers)
            offset += len(papers)
            
            # Break if we've retrieved all available papers
            if offset >= total or len(papers) < 100:
                break
                
            # Respect rate limits
            time.sleep(3)
            
        except requests.exceptions.RequestException as e:
            print(f"Error during pagination: {e}")
            time.sleep(5)
            continue
            
    return all_papers, total  # Return total for better tracking

In [5]:
def process_results(all_papers, mandatory_keywords):
    """Process and save final results with cleaning"""
    # Generate base filename with date and keywords
    current_date = datetime.now().strftime("%Y_%m_%d")
    keyword_string = "_".join(mandatory_keywords[:3]).replace(' ', '_').lower()
    base_filename = f"semantic_scholar_{current_date}_{keyword_string}"
    
    # Clean the data
    cleaned_papers = [clean_paper_data(paper) for paper in all_papers]
    
    # Convert to DataFrame
    df = pd.DataFrame(cleaned_papers)
    
    # Save main results
    results_filename = os.path.join(SAVE_DIR,f"{base_filename}_results.csv")
    df.to_csv(results_filename,
              index=False,
              sep=';',
              encoding='utf-8',
              quoting=csv.QUOTE_NONNUMERIC,
              escapechar='\\')
    
    # Process and save field frequencies
    all_fields = [field for fields in df['fieldsOfStudy'] 
                 for field in (fields if isinstance(fields, list) else [fields])]
    field_counts = Counter(all_fields)
    fields_filename = os.path.join(SAVE_DIR,f"{base_filename}_field_frequencies.csv")
    pd.DataFrame(field_counts.most_common(),
                columns=['Field of Study', 'Frequency']).to_csv(
                    fields_filename,
                    index=False,
                    sep=';',
                    encoding='utf-8',
                    quoting=csv.QUOTE_NONNUMERIC)
    
    # Process and save venue frequencies
    venue_counts = Counter(df['venue'].dropna())
    venues_filename = os.path.join(SAVE_DIR,f"{base_filename}_venue_frequencies.csv")
    pd.DataFrame(venue_counts.most_common(),
                columns=['Venue', 'Frequency']).to_csv(
                    venues_filename,
                    index=False,
                    sep=';',
                    encoding='utf-8',
                    quoting=csv.QUOTE_NONNUMERIC)

    # Process and save author frequencies
    all_authors = []
    for authors in df['authors']:
        if isinstance(authors, list):
            for author in authors:
                if isinstance(author, dict) and 'name' in author:
                    all_authors.append(author['name'])
    
    author_counts = Counter(all_authors)
    authors_filename = os.path.join(SAVE_DIR,f"{base_filename}_author_frequencies.csv")
    pd.DataFrame(author_counts.most_common(),
                columns=['Author', 'Frequency']).to_csv(
                    authors_filename,
                    index=False,
                    sep=';',
                    encoding='utf-8',
                    quoting=csv.QUOTE_NONNUMERIC)
    
    print(f"Results saved to {results_filename}")
    return df, field_counts, venue_counts, author_counts


### Some cleaning procedures

In [6]:
def clean_text_for_csv(text):
    if not isinstance(text, str):
        return text
    
    # Apply existing cleaning
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = ' '.join(text.split())
    text = text.replace('\u200b', '').replace('\ufeff', '')
    
    # Add new cleaning steps
    text = clean_ambiguous_unicode(text)
    text = normalize_unicode(text)
    
    return text
def clean_paper_data(paper):
    """Clean all text fields in a paper dictionary"""
    cleaned_paper = {}
    for key, value in paper.items():
        if isinstance(value, str):
            cleaned_paper[key] = clean_text_for_csv(value)
        elif isinstance(value, list):
            # Handle nested structures like authors and fieldsOfStudy
            cleaned_paper[key] = [
                clean_text_for_csv(item) if isinstance(item, str) 
                else item for item in value
            ]
        else:
            cleaned_paper[key] = value
    return cleaned_paper

def clean_ambiguous_unicode(text):
    if not isinstance(text, str):
        return text
        
    # Common ambiguous character replacements
    replacements = {
        '\u2018': "'",  # Left single quote
        '\u2019': "'",  # Right single quote
        '\u201C': '"',  # Left double quote
        '\u201D': '"',  # Right double quote
        '\u2013': '-',  # En dash
        '\u2014': '-',  # Em dash
    }
    
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

def normalize_unicode(text):
    if not isinstance(text, str):
        return text
    # Normalize to NFKC form (compatible composition)
    return unicodedata.normalize('NFKC', text)

def is_duplicate(paper, existing_papers):
    """Check if paper is a duplicate based on paperId"""
    return any(paper['paperId'] == existing['paperId'] for existing in existing_papers)

def merge_paper_data(existing_paper, new_paper):
    """Merge paper data, keeping the most complete information"""
    merged = existing_paper.copy()
    for key, value in new_paper.items():
        # If the new paper has information that the existing one doesn't
        if key not in existing_paper or (not existing_paper[key] and value):
            merged[key] = value
    return merged

# Search keywords and 

In [ ]:
# Define keywords and fields of study
#mandatory_keywords = ['failure rate', 'power system',' fault modelling']
mandatory_keywords =['reliability', 'resilience', 'power systems', 'capacity utilization']#, 'graph', 'neural network']
optional_keywords = []
fields_of_study = ['Computer Science', 'Engineering', 'Physics', 'Mathematics', 'Business', 'Environmental Science']
start_year=2024
end_year=2025
# Call the function to iterate through fields of study and years
all_results = iterate_fields_of_study(mandatory_keywords, optional_keywords, fields_of_study, start_year=start_year, end_year=end_year)

# Process the results
combined_df, field_freq, venue_freq, author_freq = process_results(all_results, mandatory_keywords)

print(f"Total papers retrieved: {len(combined_df)}")
print(f"Unique authors: {len(author_freq)}")

Loaded 10 existing results

Searching for year 2024
Searching for field of study: Computer Science
Searching for 2024-01, date range: 2024-01-01:2024-01-31
Month 1: 83 papers retrieved.
Searching for 2024-02, date range: 2024-02-01:2024-02-29
Month 2: 81 papers retrieved.
Searching for 2024-03, date range: 2024-03-01:2024-03-31
Month 3: 100 papers retrieved.
Searching for 2024-04, date range: 2024-04-01:2024-04-30
Month 4: 100 papers retrieved.
Searching for 2024-05, date range: 2024-05-01:2024-05-31
Month 5: 100 papers retrieved.
Searching for 2024-06, date range: 2024-06-01:2024-06-30
Month 6: 100 papers retrieved.
Searching for 2024-07, date range: 2024-07-01:2024-07-31
Rate limited. Waiting 10 seconds...
Month 7: 100 papers retrieved.
Searching for 2024-08, date range: 2024-08-01:2024-08-31
Month 8: 100 papers retrieved.
Searching for 2024-09, date range: 2024-09-01:2024-09-30
Rate limited. Waiting 10 seconds...
Month 9: 100 papers retrieved.
Searching for 2024-10, date range: 2024

: 

### New search for loadflow and contingency analysis etc. 


In [ ]:
# Define keywords and fields of study
mandatory_keywords = ['load flow','optimal load flow', 'power system','contingency analysis']#['reliability', 'resilience', 'power systems', 'capacity utilization']#, 'graph', 'neural network']
optional_keywords = ['reliability', 'resilience']
fields_of_study = ['Computer Science', 'Engineering', 'Physics', 'Mathematics', 'Business', 'Environmental Science']
start_year=2000
end_year=2024
# Call the function to iterate through fields of study and years
all_results = iterate_fields_of_study(mandatory_keywords, optional_keywords, fields_of_study, start_year=start_year, end_year=end_year)

# Process the results
combined_df, field_freq, venue_freq, author_freq = process_results(all_results, mandatory_keywords)

print(f"Total papers retrieved: {len(combined_df)}")
print(f"Unique authors: {len(author_freq)}")

Loaded 3205 existing results

Searching for year 2000
Searching for field of study: Computer Science
Date range 2000-01-01 to 2000-12-31 (offset=0): 9 papers retrieved.
Searching for field of study: Engineering
Date range 2000-01-01 to 2000-12-31 (offset=0): 30 papers retrieved.
Searching for field of study: Physics
Date range 2000-01-01 to 2000-12-31 (offset=0): 2 papers retrieved.
Searching for field of study: Mathematics
Date range 2000-01-01 to 2000-12-31 (offset=0): 5 papers retrieved.
Searching for field of study: Business
Date range 2000-01-01 to 2000-12-31 (offset=0): 0 papers retrieved.
Searching for field of study: Environmental Science
Date range 2000-01-01 to 2000-12-31 (offset=0): 8 papers retrieved.

Searching for year 2001
Searching for field of study: Computer Science
Date range 2001-01-01 to 2001-12-31 (offset=0): 13 papers retrieved.
Searching for field of study: Engineering
Date range 2001-01-01 to 2001-12-31 (offset=0): 40 papers retrieved.
Searching for field of st

### Next search for graph neural networks for power system analysis

In [ ]:
# Define keywords and fields of study
mandatory_keywords = ['Graph Neural Network','Power system analysis','Reliability analysis']#['reliability', 'resilience', 'power systems', 'capacity utilization']#, 'graph', 'neural network']
optional_keywords = ['reliability', 'resilience']
fields_of_study = ['Computer Science', 'Engineering', 'Physics', 'Mathematics', 'Business', 'Environmental Science']
start_year=2000
end_year=2025
# Call the function to iterate through fields of study and years
all_results = iterate_fields_of_study(mandatory_keywords, optional_keywords, fields_of_study, start_year=start_year, end_year=end_year)

# Process the results
combined_df, field_freq, venue_freq, author_freq = process_results(all_results, mandatory_keywords)

print(f"Total papers retrieved: {len(combined_df)}")
print(f"Unique authors: {len(author_freq)}")

Starting fresh search

Searching for year 2000
Searching for field of study: Computer Science
Date range 2000-01-01 to 2000-12-31 (offset=0): 7 papers retrieved.
Searching for field of study: Engineering
Date range 2000-01-01 to 2000-12-31 (offset=0): 5 papers retrieved.
Searching for field of study: Physics
Rate limited. Waiting 10 seconds...
Date range 2000-01-01 to 2000-12-31 (offset=0): 0 papers retrieved.
Searching for field of study: Mathematics
Rate limited. Waiting 10 seconds...
Rate limited. Waiting 10 seconds...
Rate limited. Waiting 10 seconds...
Failed to retrieve papers for 2000-01-01 to 2000-12-31 after 3 attempts.
Searching for field of study: Business
Rate limited. Waiting 10 seconds...
Date range 2000-01-01 to 2000-12-31 (offset=0): 0 papers retrieved.
Searching for field of study: Environmental Science
Rate limited. Waiting 10 seconds...
Date range 2000-01-01 to 2000-12-31 (offset=0): 2 papers retrieved.

Searching for year 2001
Searching for field of study: Computer 

### searching for specific keywords such as probabilistic analysis, Loss of load probabilit, monte carlo etc.

In [ ]:
# Define keywords and fields of study
mandatory_keywords = ['contingency analysis','capacity utilization','reliability', 'power systems', 'power grid']#, 'graph', 'neural network']
optional_keywords = ['reliability','stochastic', 'resilience','resilience analysis', 'loss of load', 'contingency analysis','capacity utilization']
fields_of_study = ['Computer Science', 'Engineering', 'Physics', 'Mathematics', 'Business', 'Environmental Science']
start_year=2010
end_year=2025
# Call the function to iterate through fields of study and years
all_results = iterate_fields_of_study(mandatory_keywords, optional_keywords, fields_of_study, start_year=start_year, end_year=end_year)

# Process the results
combined_df, field_freq, venue_freq, author_freq = process_results(all_results, mandatory_keywords)

print(f"Total papers retrieved: {len(combined_df)}")
print(f"Unique authors: {len(author_freq)}")

Loaded 10 existing results

Searching for year 2010
Searching for field of study: Computer Science
Date range 2010-01-01 to 2010-12-31 (offset=0): 0 papers retrieved.
Searching for field of study: Engineering
Rate limited. Waiting 10 seconds...
Rate limited. Waiting 10 seconds...
Date range 2010-01-01 to 2010-12-31 (offset=0): 1 papers retrieved.
Searching for field of study: Physics
Date range 2010-01-01 to 2010-12-31 (offset=0): 0 papers retrieved.
Searching for field of study: Mathematics
Rate limited. Waiting 10 seconds...
Rate limited. Waiting 10 seconds...
Rate limited. Waiting 10 seconds...
Failed to retrieve papers for 2010-01-01 to 2010-12-31 after 3 attempts.
Searching for field of study: Business
Date range 2010-01-01 to 2010-12-31 (offset=0): 0 papers retrieved.
Searching for field of study: Environmental Science
Rate limited. Waiting 10 seconds...
Rate limited. Waiting 10 seconds...
Date range 2010-01-01 to 2010-12-31 (offset=0): 1 papers retrieved.

Searching for year 201

### When search is interrupted:

In [ ]:
def convert_partial_results_to_csv():
    """Convert partial results from JSON to CSV with proper cleaning"""
    try:
        # Load partial results
        with open("partial_results.json", 'r', encoding='utf-8') as f:
            papers = json.load(f)
        
        if not papers:
            print("No papers found in partial results")
            return
            
        # Clean the data
        cleaned_papers = [clean_paper_data(paper) for paper in papers]
        
        # Convert to DataFrame
        df = pd.DataFrame(cleaned_papers)
        
        # Save main results with proper encoding and escaping
        df.to_csv("semantic_scholar_results.csv", 
                  index=False, 
                  sep=';',
                  encoding='utf-8',
                  quoting=csv.QUOTE_NONNUMERIC,
                  escapechar='\\')
        
        # Process field frequencies
        all_fields = [field for fields in df['fieldsOfStudy'] 
                     for field in (fields if isinstance(fields, list) else [fields])]
        field_counts = Counter(all_fields)
        
        # Save field frequencies
        pd.DataFrame(field_counts.most_common(), 
                    columns=['Field of Study', 'Frequency']).to_csv(
                        "field_of_study_frequencies.csv",
                        index=False,
                        sep=';',
                        encoding='utf-8',
                        quoting=csv.QUOTE_NONNUMERIC)
        
        # Process venue frequencies
        venue_counts = Counter(df['venue'].dropna())
        
        # Save venue frequencies
        pd.DataFrame(venue_counts.most_common(), 
                    columns=['Venue', 'Frequency']).to_csv(
                        "venue_frequencies.csv",
                        index=False,
                        sep=';',
                        encoding='utf-8',
                        quoting=csv.QUOTE_NONNUMERIC)
        
        print(f"Successfully converted {len(papers)} papers to CSV format")
        
    except Exception as e:
        print(f"Error converting partial results: {e}")
convert_partial_results_to_csv()

Successfully converted 18405 papers to CSV format


In [ ]:
# Define keywords and fields of study
mandatory_keywords = ['reliability', 'resilience', 'power systems', 'capacity utilization', 'graph','neural network']
optional_keywords = []
fields_of_study = ['Computer Science', 'Engineering', 'Physics', 'Mathematics', 'Business', 'Environmental Science']
# Now call the function to iterate through fields of study and years

all_results = iterate_fields_of_study(mandatory_keywords, optional_keywords, fields_of_study)

# Process the results
combined_df, field_freq, venue_freq = process_results(all_results)

# Save results to CSV files
combined_df.to_csv("semantic_scholar_results.csv", index=False)
pd.DataFrame(field_freq.most_common(), columns=['Field of Study', 'Frequency']).to_csv("field_of_study_frequencies.csv", index=False)
pd.DataFrame(venue_freq.most_common(), columns=['Venue', 'Frequency']).to_csv("venue_frequencies.csv", index=False)

print(f"Total papers retrieved: {len(combined_df)}")
print("Results saved to CSV files.")


Searching for year 1980
Searching for field of study: Computer Science
Searching papers for year 1980, field of study Computer Science, with query: "reliability" "resilience" "power systems" "capacity utilization" "graph" "neural network", offset: 0
An error occurred while searching papers: 403 Client Error: Forbidden for url: https://api.semanticscholar.org/graph/v1/paper/search?query=%22reliability%22+%22resilience%22+%22power+systems%22+%22capacity+utilization%22+%22graph%22+%22neural+network%22&year=1980&fields=paperId%2Ctitle%2Cabstract%2Cauthors%2Cyear%2CcitationCount%2CfieldsOfStudy%2Cvenue&limit=50&offset=0&fieldsOfStudy=Computer+Science
Searching for field of study: Engineering
Searching papers for year 1980, field of study Engineering, with query: "reliability" "resilience" "power systems" "capacity utilization" "graph" "neural network", offset: 0
An error occurred while searching papers: 403 Client Error: Forbidden for url: https://api.semanticscholar.org/graph/v1/paper/sea

KeyboardInterrupt: 

Script started.
Search query: "reliability"+"resilience"+"power systems"+"capacity utilization"

Searching for year 2000
Searching papers for year 2000 with query: "reliability"+"resilience"+"power systems"+"capacity utilization", offset: 0
Number of papers retrieved for year 2000: 100
Searching papers for year 2000 with query: "reliability"+"resilience"+"power systems"+"capacity utilization", offset: 100
Number of papers retrieved for year 2000: 100
Searching papers for year 2000 with query: "reliability"+"resilience"+"power systems"+"capacity utilization", offset: 200
Number of papers retrieved for year 2000: 13
Total papers retrieved for year 2000: 213

Searching for year 2001
Searching papers for year 2001 with query: "reliability"+"resilience"+"power systems"+"capacity utilization", offset: 0
Number of papers retrieved for year 2001: 100
Searching papers for year 2001 with query: "reliability"+"resilience"+"power systems"+"capacity utilization", offset: 100
Number of papers retrie

In [ ]:
all_fields = []
for fields in df['fieldsOfStudy']:
    if isinstance(fields, list):
        all_fields.extend(fields)
    elif isinstance(fields, str):
        all_fields.append(fields)
    
field_counts = Counter(all_fields)

print("\nField of Study Frequencies:")
for field, count in field_counts.most_common():
    print(f"{field}: {count}")


Field of Study Frequencies:
Computer Science: 884
Engineering: 359
Mathematics: 33
Physics: 13
Medicine: 13
Economics: 6
Business: 5
Environmental Science: 4
Materials Science: 2
Chemistry: 1
Psychology: 1


In [ ]:

def count_venue_frequencies(df):
    if 'venue' in df.columns:
        # Count frequency of venues
        all_venues = df['venue'].dropna().tolist()  # Drop NA values from the venue column
        venue_counts = Counter(all_venues)
        
        # Create a DataFrame with venue frequencies
        venue_freq_df = pd.DataFrame(venue_counts.most_common(), columns=['Venue', 'Frequency'])
    else:
        print("Warning: 'venue' column not found in the DataFrame.")
        # Create an empty DataFrame if 'venue' is not present
        venue_freq_df = pd.DataFrame(columns=['Venue', 'Frequency'])
    
    return venue_freq_df

# Assuming 'df' is your existing DataFrame with the search results
# If it's not named 'df', replace 'df' with the actual name of your DataFrame

# Count venue frequencies
venue_freq_df = count_venue_frequencies(df)

if not venue_freq_df.empty:
    # Display the top 20 most frequent venues
    print("\nTop 20 Most Frequent Venues:")
    print(venue_freq_df.head(20))

    # Save venue frequencies to CSV
    venue_freq_df.to_csv("venue_frequencies.csv", index=False)
    print("Venue frequencies saved to venue_frequencies.csv")
else:
    print("No venue information available.")

# If you want to add the venue frequency to your existing field of study frequency DataFrame
try:
    field_freq_df = pd.read_csv("field_of_study_frequencies.csv")  # Load existing field frequencies
    if not venue_freq_df.empty:
        combined_freq_df = pd.concat([field_freq_df, venue_freq_df], axis=1)
    else:
        combined_freq_df = field_freq_df
    combined_freq_df.to_csv("combined_frequencies.csv", index=False)
    print("Combined frequencies saved to combined_frequencies.csv")
except FileNotFoundError:
    print("field_of_study_frequencies.csv not found. Skipping combined frequencies.")

No venue information available.
Combined frequencies saved to combined_frequencies.csv
